In [105]:
#LIBRARIES
import pandas as pd
import numpy as np
import quandl
from dotenv import load_dotenv
import os
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegresion

In [120]:
# GETS THE API KEY FROM .ENV FILE
def get_key(d_key):
    load_dotenv('.env')
    key = os.environ.get(d_key)
    return key

# GETS DATA FROM QUANDL / DEFAULT IS BOING DATA 
def get_data(code="EOD/BA", start_date="1970-01-01", end_date="2019-05-27"):
    return quandl.get(code, start_date=start_date, end_date=end_date)

# KEEP SIGNIFICANT COLUMNS. ADD CHANGE AND VOLATILITY
def feature_eng(df, n=10):
    
    df['Volatility'] = (df['Adj_High'] - data['Adj_Low']) / data['Adj_Low']
    df['Change'] = (df['Adj_Close'] - df['Adj_Open']) / df['Adj_Open']
    df=df[['Adj_Close', 'Volatility', 'Change', 'Adj_Volume']]
    
    #Defines forecast to predict
    forecast_column = 'Adj_Close'
    df.fillna('-999999', inplace=True) # N/A value treated as outlier
    forecast_out = n # number of days of prediction
    
    #shifts label
    df['future_price'] = df[forecast_column].shift(-forecast_out)
    df.dropna(inplace=True)
    return df

In [116]:
quandl_key = get_key('quandl_key')
quandl.ApiConfig.api_key = quandl_key
data = get_data()
data.tail()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2017-12-21,298.040,298.3852,294.1301,295.03,5395193.0,0.0,1.0,289.075230,289.410047,285.282937,286.155768,5395193.0
2017-12-22,295.135,296.5200,293.0100,295.10,2257000.0,0.0,1.0,286.257610,287.600951,284.196528,286.223663,2257000.0
2017-12-26,295.540,297.4400,294.6500,295.36,1394737.0,0.0,1.0,286.650428,288.493278,285.787198,286.475842,1394737.0
2017-12-27,296.410,296.8000,294.8000,295.62,1447837.0,0.0,1.0,287.494259,287.872528,285.932687,286.728022,1447837.0
2017-12-28,295.810,296.9900,294.7400,296.35,1436543.0,0.0,1.0,286.912307,288.056813,285.874491,287.436064,1436543.0


In [121]:
data_ready = feature_eng(data)
data_ready.tail(30)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Adj_Close,Volatility,Change,Adj_Volume,future_price
Date,,,,,
2017-11-01,249.376576,0.012371,0.000813,2929490.0,254.953412
2017-11-02,253.360813,0.020310,0.016724,2528457.0,255.768146
2017-11-03,252.511871,0.014265,-0.002553,2610784.0,254.371460
2017-11-06,254.749990,0.012598,0.005521,2909813.0,256.670172
2017-11-07,256.737285,0.012156,0.007801,3205378.0,258.959186
2017-11-08,256.197049,0.008783,-0.002142,2722230.0,257.174531
2017-11-09,254.798225,0.027690,-0.008717,3215203.0,257.882574
2017-11-10,253.003871,0.011171,-0.004807,1740271.0,257.591597
2017-11-13,254.526647,0.011018,0.002866,2604815.0,259.929107


In [122]:
X = data_ready.drop('future_price', axis=1)
X = preprocessing.scale(X)
y = data_ready['future_price']

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.05)